In [63]:
import numpy as np

In [64]:
M = np.zeros((6,17))

In [65]:
S1 = [[0,7,12,15],
     [0,6,13],
     [3,9,13],
     [3,6,9,12,15],
     [3,6,11,14,16],
     [5,9,14,16]]
S2 = S1

In [66]:
for j in range(0,6):
    for i in range(0,len(S[j])):
        M[j][S1[j][i]] = 1

In [67]:
M

array([[1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
        0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
        0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
        1.]])

At this point matrix M contains the schedule of available match days where the only valid slots are where the value is 0

In [68]:
G1 = np.zeros((4,6))
for k in range(0,4):
    for j in range(0,6):
        G1[k][j] = 5*k + j + k + 1

G2 = G1
G1

array([[ 1.,  2.,  3.,  4.,  5.,  6.],
       [ 7.,  8.,  9., 10., 11., 12.],
       [13., 14., 15., 16., 17., 18.],
       [19., 20., 21., 22., 23., 24.]])

In [69]:
within_3_days = np.array([[2,3,4,5],[1,3,4,6],[1,2,5,6],[1,2,5,6],[1,3,4,6],[2,3,4,5],
                [8,9,10,11],[7,9,10,12],[7,8,11,12],[7,8,11,12],[7,9,10,12],[8,9,10,11],
                 [14,15,16,17],[13,15,16,18],[13,14,17,18],[13,14,17,18],[13,15,16,18],[14,15,16,17],
                 [20,21,22,23],[19,21,22,24],[19,20,23,24],[19,20,23,24],[19,21,22,24],[20,21,22,23]])
#this shows which matches cannot be within 3 days of each other. 
#for a given match i within_3_days[i] gives a list of the matches that cannot be within 3 days of match i.

In [70]:
scheduled = -M
scheduled[1][0] = 1
scheduled[0][7] = 2
scheduled[1][13] = 3
scheduled
#this code assigned matches for Mexico that have already been predetermined by FIFA

array([[-1., -0., -0., -0., -0., -0., -0.,  2., -0., -0., -0., -0., -1.,
        -0., -0., -1., -0.],
       [ 1., -0., -0., -0., -0., -0., -1., -0., -0., -0., -0., -0., -0.,
         3., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -0., -0., -0., -1., -0., -0., -0.,
        -1., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -1., -0., -0., -1., -0., -0., -1.,
        -0., -0., -1., -0.],
       [-0., -0., -0., -1., -0., -0., -1., -0., -0., -0., -0., -1., -0.,
        -0., -1., -0., -1.],
       [-0., -0., -0., -0., -0., -1., -0., -0., -0., -1., -0., -0., -0.,
        -0., -1., -0., -1.]])

In [71]:
def check_constraints(day,stadium,match,scheduled):
    if((day in S1[stadium]) and (day in S2[stadium])): #checks if the stadium contains a slot for the given day and removes that slot if valid
        S2[stadium].pop(day)
        return True
    else: return False
    if(not check_3_days(match,day,scheduled)): #checks if the given match allows 3 days of rest
        return False
    else: return True

In [73]:
def check_3_days(match,day,scheduled):
    illegal = within_3_days[match]
    if((day-3)<=0):
        start = 0
    else: start = day-3
    if((day+3)>=23):
        end = 23
    else: end = day + 3
    for i in range(0,len(illegal)):
        if(i in scheduled[:,start:end]):
            return False
    return True

In [74]:
#pseudo-code for the actual solver

#solver(S1,S2,M,scheduled,G1,G2):
    #choose a match from G1 that is still available in G2 and pick the first valid slot from M (i.e. extract indeces of M)
    #so for the chosen slot M: stadium = row i, day equals column j
    #if(check_constraints(j,i,G1 choice,scheduled))
        #G2.pop(G1 choice)
        #scheduled[stadium][day] = match
    #else backtrack
    #call solver again with new scheduled